In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

In [2]:
# Dataset de StephenKing
# The Shining
file_path = "./StephenKing_books/Stephen_King_TheShining.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    book = file.read()

In [3]:
# Preprocesamiento con spacy ya que le dataset
# es texto denso sin signos y no esta divido en oraciones
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(book)
sentences = [sent.text for sent in doc.sents] # separamos por oraciones

In [4]:
# Visualizamos documentos en formato dataframe
df = pd.DataFrame(sentences)
df.head(10)

,0
0,This is for Joe Hill King who shines on My edi...
1,Some of the most beautiful resort hotels in th...
2,rang it was observed that the giddiest grew pa...
3,But when the echoes had fully ceased a light l...
4,E A POE The Masque of the Red Death
5,The sleep of reason breeds monsters GOYA
6,Itll shine when it shines FOLK SAYING Part One...
7,This had better be good you There was a red ca...
8,sorry I asked if your wife fully understood wh...
9,And theres your son of course He glanced down ...


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 3184


In [6]:
# continuamos preprocesamiento con Keras para reducir pasos de limpiar texto
# Recorrer todas las filas y transformar las oraciones en una secuencia de palabras
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [64]:
sentence_tokens[4]

['e', 'a', 'poe', 'the', 'masque', 'of', 'the', 'red', 'death']

### Creamos vectores (Word2Vec) 

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

### Usamos CBOW

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model_cbow = Word2Vec(min_count=10,    # frecuencia mínima de palabra para incluirla en el vocabulario
                          window=2,       # cant de palabras antes y desp de la predicha
                          vector_size=200,       # dimensionalidad de los vectores 
                          negative=30,    # cantidad de negative samples... 0 es no se usa
                          workers=1,      # si tienen más cores pueden cambiar este valor
                          sg=0)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model_cbow.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_cbow.corpus_count)
# Cantidad de words encontradas en el corpus
print("Cantidad de palabras distintas en el corpus:", len(w2v_model_cbow.wv.index_to_key))

Cantidad de docs en el corpus: 3184
Cantidad de palabras distintas en el corpus: 1609


### Entrenar embeddings 

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_cbow.train(sentence_tokens,
                     total_examples=w2v_model_cbow.corpus_count,
                     epochs=100,
                     compute_loss = True,
                     callbacks=[callback()]
                     )

Loss after epoch 0: 527371.6875
Loss after epoch 1: 415514.9375
Loss after epoch 2: 359852.125
Loss after epoch 3: 338593.0
Loss after epoch 4: 333357.5
Loss after epoch 5: 313782.5
Loss after epoch 6: 299610.25
Loss after epoch 7: 296866.75
Loss after epoch 8: 292653.5
Loss after epoch 9: 288904.0
Loss after epoch 10: 286467.25
Loss after epoch 11: 282246.75
Loss after epoch 12: 271484.25
Loss after epoch 13: 259190.0
Loss after epoch 14: 256163.5
Loss after epoch 15: 253931.5
Loss after epoch 16: 250922.5
Loss after epoch 17: 249266.5
Loss after epoch 18: 247022.5
Loss after epoch 19: 244554.0
Loss after epoch 20: 242661.0
Loss after epoch 21: 241662.0
Loss after epoch 22: 239163.5
Loss after epoch 23: 237234.0
Loss after epoch 24: 235124.5
Loss after epoch 25: 232549.5
Loss after epoch 26: 232647.0
Loss after epoch 27: 231168.5
Loss after epoch 28: 229596.5
Loss after epoch 29: 226932.5
Loss after epoch 30: 217072.0
Loss after epoch 31: 216492.0
Loss after epoch 32: 215334.0
Loss af

(9455568, 16309300)

### Evaluación

In [13]:
# Palabras que MÁS se relacionan con...:
w2v_model_cbow.wv.most_similar(positive=["monsters"], topn=10)

[('redrum', 0.3837184011936188),
 ('pounding', 0.37854453921318054),
 ('makes', 0.3435423672199249),
 ('cabinet', 0.33662939071655273),
 ('beyond', 0.33180898427963257),
 ('noise', 0.32834726572036743),
 ('awful', 0.3103261888027191),
 ('kind', 0.3061906695365906),
 ('muffled', 0.29882076382637024),
 ('lines', 0.28683823347091675)]

In [14]:
# Palabras que Menos se relacionan con...:
w2v_model_cbow.wv.most_similar(negative=["monsters"], topn=10)

[('walking', 0.281059205532074),
 ('folks', 0.2758888006210327),
 ('youve', 0.26370149850845337),
 ('friends', 0.2614437937736511),
 ('jacky', 0.2571820914745331),
 ('wife', 0.25360968708992004),
 ('steps', 0.2260364294052124),
 ('twelve', 0.21330417692661285),
 ('harder', 0.20936912298202515),
 ('youll', 0.20380716025829315)]

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model_cbow.wv.most_similar(positive=["ghost"], topn=10)

[('animal', 0.31758806109428406),
 ('funny', 0.28930795192718506),
 ('figure', 0.26031845808029175),
 ('lot', 0.24839335680007935),
 ('trouble', 0.23688240349292755),
 ('having', 0.23224827647209167),
 ('theres', 0.23146969079971313),
 ('certainly', 0.22833122313022614),
 ('full', 0.22530555725097656),
 ('different', 0.22289533913135529)]

In [16]:
# Palabras que Menos se relacionan con...:
w2v_model_cbow.wv.most_similar(negative=["ghost"], topn=10)

[('stared', 0.27145329117774963),
 ('whirled', 0.2677803635597229),
 ('ran', 0.2627612352371216),
 ('shifted', 0.25799986720085144),
 ('stumbled', 0.25789520144462585),
 ('collapsed', 0.2463955134153366),
 ('continued', 0.24310851097106934),
 ('promised', 0.2421831488609314),
 ('glanced', 0.2414068728685379),
 ('onto', 0.2364109754562378)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model_cbow.wv.most_similar(positive=["danny"], topn=10)

[('jack', 0.4968011975288391),
 ('he', 0.48612746596336365),
 ('she', 0.44337594509124756),
 ('wendy', 0.4303872287273407),
 ('hallorann', 0.40921124815940857),
 ('daddy', 0.3704814016819),
 ('edmonds', 0.35048770904541016),
 ('him', 0.34431686997413635),
 ('tony', 0.3400289714336395),
 ('mother', 0.33276766538619995)]

In [18]:
# Palabras que Menos se relacionan con...:
w2v_model_cbow.wv.most_similar(negative=["danny"], topn=10)

[('range', 0.2929755449295044),
 ('of', 0.2820560336112976),
 ('building', 0.2636813521385193),
 ('west', 0.26235267519950867),
 ('marked', 0.2459445595741272),
 ('thousand', 0.23162025213241577),
 ('human', 0.23076224327087402),
 ('written', 0.23075534403324127),
 ('struck', 0.22104084491729736),
 ('hissing', 0.2194625586271286)]

In [32]:
# el método `get_vector` permite obtener los vectores:
vector_danny = w2v_model_cbow.wv.get_vector("danny")
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_cbow.wv.most_similar(vector_danny)

[('danny', 1.0),
 ('jack', 0.4968011975288391),
 ('he', 0.48612746596336365),
 ('she', 0.44337594509124756),
 ('wendy', 0.4303872287273407),
 ('hallorann', 0.40921130776405334),
 ('daddy', 0.3704814016819),
 ('edmonds', 0.35048770904541016),
 ('him', 0.34431689977645874),
 ('tony', 0.3400289714336395)]

In [37]:
# el método `get_vector` permite obtener los vectores:
vector_hotel = w2v_model_cbow.wv.get_vector("hotel")
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_cbow.wv.most_similar(vector_hotel)

[('hotel', 1.0),
 ('overlook', 0.28946220874786377),
 ('town', 0.279313862323761),
 ('wing', 0.2580367624759674),
 ('place', 0.2465081810951233),
 ('hotels', 0.24398307502269745),
 ('roads', 0.2321082204580307),
 ('fire', 0.22380448877811432),
 ('trouble', 0.21784675121307373),
 ('window', 0.21416300535202026)]

In [38]:
# el método `get_vector` permite obtener los vectores:
vector_shining = w2v_model_cbow.wv.get_vector("shining")
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_cbow.wv.most_similar(vector_shining)

[('shining', 1.0),
 ('crash', 0.31426092982292175),
 ('picture', 0.254147469997406),
 ('yeah', 0.24933332204818726),
 ('understood', 0.24299338459968567),
 ('story', 0.23741285502910614),
 ('gets', 0.23212751746177673),
 ('lion', 0.22786845266819),
 ('riding', 0.22074149549007416),
 ('hedge', 0.2204970270395279)]

In [39]:
# el método `get_vector` permite obtener los vectores:
vector_shine = w2v_model_cbow.wv.get_vector("shine")
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_cbow.wv.most_similar(vector_shine)

[('shine', 1.0),
 ('know', 0.39431533217430115),
 ('worry', 0.3458254933357239),
 ('happened', 0.29749035835266113),
 ('talk', 0.2844362258911133),
 ('wake', 0.2786942720413208),
 ('pants', 0.27759525179862976),
 ('sometimes', 0.2772965729236603),
 ('tell', 0.2758985161781311),
 ('knows', 0.2758241295814514)]

### Visualizamos agrupaciones de vectores

In [19]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [29]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model_cbow)

MAX_WORDS=300
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() 

In [30]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model_cbow,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show()

**Analisis**

Como se observa se hizo un embedding utilizando Gensim y el corpus fue del famoso libro de Stephen King - The Shining. Para el embedding se utilizo Word2Vec con el algoritmo de CBOW, donde, se entreno con un total de 100 epocas, un corpus 3184 y 1609 palabras en el vocabulario.

Se analizo el corpus y se buscaron esas palabras que se agrupan por su similitud. Entre los mejores resultados tenemos las siguientes:

* "danny" - que como sabemos es el nombre de uno de los personajes que hace referencia al niño que tenía la habilidad (shine). Se puede observar que CBOW es capaz de agruparlo por similitud con los nombres de otros personajes con 

1. ('jack', 0.4968011975288391),
2. ('he', 0.48612746596336365),
3. ('she', 0.44337594509124756),
4. ('wendy', 0.4303872287273407),
5. ('hallorann', 0.40921124815940857),
6. ('daddy', 0.3704814016819),
7. ('edmonds', 0.35048770904541016),
8. ('him', 0.34431686997413635),
9. ('tony', 0.3400289714336395),
10. ('mother', 0.33276766538619995)

Comparandolo a nivel de vectores vemos un resultado muy parecido

1. ('danny', 1.0),
2. ('jack', 0.4968011975288391),
3. ('he', 0.48612746596336365),
4. ('she', 0.44337594509124756),
5. ('wendy', 0.4303872287273407),
6. ('hallorann', 0.40921130776405334),
7. ('daddy', 0.3704814016819),
8. ('edmonds', 0.35048770904541016),
9. ('him', 0.34431689977645874),
10. ('tony', 0.3400289714336395)

* "hotel" - es otra de las palabras que CBOW es capaz de asociar bien. Sabiendo que es el lugar donde se desarrolla la historia observamos que las palabras que agrupa el algoritmo son palabras que describen en si el hotel.

1. ('hotel', 1.0),
2. ('overlook', 0.28946220874786377),
3. ('town', 0.279313862323761),
4. ('wing', 0.2580367624759674),
5. ('place', 0.2465081810951233),
6. ('hotels', 0.24398307502269745),
7. ('roads', 0.2321082204580307),
7. ('fire', 0.22380448877811432),
8. ('trouble', 0.21784675121307373),
9. ('window', 0.21416300535202026)

* "Shine" y "Shining" - Tambien se considera que tuvo una buena agrupación por parte del algoritmo. Recordando una de las frases famosas:

"People who shine can sometimes see things that are gonna happen, and I think sometimes they can see things that did happen. But they’re just like pictures in a book".

vemos que nos da el vector "Shine":

1. ('shine', 1.0),
2. ('know', 0.39431533217430115),
3. ('worry', 0.3458254933357239),
4. ('happened', 0.29749035835266113),
5. ('talk', 0.2844362258911133),
6. ('wake', 0.2786942720413208),
7. ('pants', 0.27759525179862976),
8. ('sometimes', 0.2772965729236603),
9. ('tell', 0.2758985161781311),
10. ('knows', 0.2758241295814514)

vemos que nos da el vector "Shining":

1. ('shining', 1.0),
2. ('crash', 0.31426092982292175),
3. ('picture', 0.254147469997406),
4. ('yeah', 0.24933332204818726),
5. ('understood', 0.24299338459968567),
6. ('story', 0.23741285502910614),
7. ('gets', 0.23212751746177673),
8. ('lion', 0.22786845266819),
9. ('riding', 0.22074149549007416),
10. ('hedge', 0.2204970270395279)

Reduciendo a 2 dimensiones con TSNE se puede observar agrupaciones muy parecidas al momento de graficarlas en 2D y en 3D.

Ahora probaremos las diferencias con skipgram.

### Con Skip-gram

In [40]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model_skip = Word2Vec(min_count=10,    # frecuencia mínima de palabra para incluirla en el vocabulario
                          window=2,       # cant de palabras antes y desp de la predicha
                          vector_size=200,       # dimensionalidad de los vectores 
                          negative=30,    # cantidad de negative samples... 0 es no se usa
                          workers=1,      # si tienen más cores pueden cambiar este valor
                          sg=1)           # modelo 0:CBOW  1:skipgram

In [42]:
# Obtener el vocabulario con los tokens
w2v_model_skip.build_vocab(sentence_tokens)
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_skip.corpus_count)
# Cantidad de words encontradas en el corpus
print("Cantidad de palabras distintas en el corpus:", len(w2v_model_skip.wv.index_to_key))

Cantidad de docs en el corpus: 3184
Cantidad de palabras distintas en el corpus: 1609


In [43]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_skip.train(sentence_tokens,
                     total_examples=w2v_model_skip.corpus_count,
                     epochs=100,
                     compute_loss = True,
                     callbacks=[callback()]
                     )

Loss after epoch 0: 1278796.0
Loss after epoch 1: 968853.0
Loss after epoch 2: 896639.5
Loss after epoch 3: 887496.0
Loss after epoch 4: 855720.5
Loss after epoch 5: 843768.5
Loss after epoch 6: 836997.0
Loss after epoch 7: 834485.5
Loss after epoch 8: 827252.5
Loss after epoch 9: 801313.5
Loss after epoch 10: 795642.0
Loss after epoch 11: 787041.0
Loss after epoch 12: 780698.0
Loss after epoch 13: 778616.0
Loss after epoch 14: 772387.0
Loss after epoch 15: 767671.0
Loss after epoch 16: 764182.0
Loss after epoch 17: 762249.0
Loss after epoch 18: 757942.0
Loss after epoch 19: 753234.0
Loss after epoch 20: 734772.0
Loss after epoch 21: 732246.0
Loss after epoch 22: 725888.0
Loss after epoch 23: 726386.0
Loss after epoch 24: 721374.0
Loss after epoch 25: 715144.0
Loss after epoch 26: 717236.0
Loss after epoch 27: 715706.0
Loss after epoch 28: 713530.0
Loss after epoch 29: 708162.0
Loss after epoch 30: 706644.0
Loss after epoch 31: 707634.0
Loss after epoch 32: 706134.0
Loss after epoch 33

(9455568, 16309300)

In [47]:
# Palabras que MÁS se relacionan con...:
w2v_model_skip.wv.most_similar(positive=["danny"], topn=10)

[('he', 0.46182015538215637),
 ('jack', 0.39433926343917847),
 ('it', 0.3686791956424713),
 ('him', 0.3543377220630646),
 ('she', 0.3417852520942688),
 ('wendy', 0.3271055817604065),
 ('and', 0.32683825492858887),
 ('hey', 0.3267696797847748),
 ('said', 0.32201066613197327),
 ('yes', 0.31284552812576294)]

In [48]:
# Palabras que MÁS se relacionan con...:
w2v_model_skip.wv.most_similar(positive=["monsters"], topn=10)

[('pounding', 0.41447779536247253),
 ('opening', 0.39405521750450134),
 ('fear', 0.3847183585166931),
 ('shingles', 0.34984269738197327),
 ('makes', 0.34771138429641724),
 ('animal', 0.34745293855667114),
 ('cabinet', 0.3455350995063782),
 ('noise', 0.34542784094810486),
 ('mallet', 0.34138941764831543),
 ('booming', 0.3334644138813019)]

In [49]:
# Palabras que MÁS se relacionan con...:
w2v_model_skip.wv.most_similar(positive=["ghost"], topn=10)

[('dressed', 0.37305986881256104),
 ('older', 0.36076244711875916),
 ('fit', 0.3384249210357666),
 ('pieces', 0.3226546347141266),
 ('strange', 0.3153647780418396),
 ('havent', 0.3105677366256714),
 ('figure', 0.31036993861198425),
 ('closet', 0.30844971537590027),
 ('knows', 0.30795755982398987),
 ('bit', 0.30787134170532227)]

In [51]:
# el método `get_vector` permite obtener los vectores:
vector_danny = w2v_model_skip.wv.get_vector("danny")
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_skip.wv.most_similar(vector_danny)

[('danny', 0.9999999403953552),
 ('he', 0.46182015538215637),
 ('jack', 0.39433926343917847),
 ('it', 0.3686792254447937),
 ('him', 0.3543377220630646),
 ('she', 0.3417852520942688),
 ('wendy', 0.3271056115627289),
 ('and', 0.32683825492858887),
 ('hey', 0.32676970958709717),
 ('said', 0.32201069593429565)]

In [52]:
# el método `get_vector` permite obtener los vectores:
vector_hotel = w2v_model_skip.wv.get_vector("hotel")
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_skip.wv.most_similar(vector_hotel)

[('hotel', 1.0),
 ('miles', 0.31923189759254456),
 ('california', 0.29975125193595886),
 ('attic', 0.2970217764377594),
 ('overlook', 0.2909839451313019),
 ('truck', 0.28776901960372925),
 ('curb', 0.28609079122543335),
 ('says', 0.28508779406547546),
 ('hotels', 0.269979864358902),
 ('deserted', 0.26480311155319214)]

In [53]:
# el método `get_vector` permite obtener los vectores:
vector_shining = w2v_model_skip.wv.get_vector("shining")
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_skip.wv.most_similar(vector_shining)

[('shining', 1.0),
 ('yeah', 0.3078247308731079),
 ('riding', 0.29755479097366333),
 ('pencil', 0.2831701934337616),
 ('roger', 0.28232306241989136),
 ('wrong', 0.28060251474380493),
 ('understood', 0.28035518527030945),
 ('did', 0.27980566024780273),
 ('crash', 0.27871325612068176),
 ('quarter', 0.27858853340148926)]

In [54]:
# el método `get_vector` permite obtener los vectores:
vector_shine = w2v_model_skip.wv.get_vector("shine")
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_skip.wv.most_similar(vector_shine)

[('shine', 0.9999999403953552),
 ('anyone', 0.3630369305610657),
 ('worry', 0.35454750061035156),
 ('wake', 0.34417858719825745),
 ('pants', 0.32322144508361816),
 ('happened', 0.3192065358161926),
 ('plane', 0.3176001310348511),
 ('answered', 0.31268855929374695),
 ('accident', 0.3011113703250885),
 ('known', 0.29653608798980713)]

In [55]:
vecs, labels = reduce_dimensions(w2v_model_skip)

MAX_WORDS=300
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() 

**Analisis**

Probando con skip-gram es notable que hay una diferencia en comparación con CBOW. Por ejemplo, para palabras como "ghost" y "monsters" que CBOW tuvo una no tan buena relación, Skip-gram logro crear un mejor match.

Si bien muchas palabras en ambas algoritmos se agrupan de forma similar es notable que hay una diferencia incluso en la visualización de los datos en 2D.

En general, ambos algoritmos lograron agrupar terminos similares según el contexto de forma satisfactoria, dando una buena idea de los sucesos que transcurren durante la historia solo en base a la relación de terminos.